In [1]:
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
config = PeftConfig.from_pretrained("bogeumkim/polyglot-1.3b-qlora-emotion-classification")

BASE_MODEL = "EleutherAI/polyglot-ko-1.3b"
PEFT_MODEL = "bogeumkim/polyglot-1.3b-qlora-emotion-classification"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL)
peft_model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

In [4]:
model.eval()

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30080, 2048)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(
            in_features=2048, out_features=6144, bias=True
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_featu

In [19]:
def gen(x):
    q = f"### 질문: {x}\n### 답변:"

    gened = model.generate(
        **tokenizer(
            q, 
            return_tensors='pt', 
            return_token_type_ids=False
        ),
        
        max_new_tokens=3,
        temperature=0.001,
        do_sample=True,
        eos_token_id=2,
    )

    result = tokenizer.decode(gened[0]).split('\n')[1]
    result = result.split(':')[1]

    return result.strip()

In [20]:
gen('친구가 따돌림을 당하는 걸 방관하고 있는데 진짜 마음이 너무 괴로워.')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'자살충동'

In [21]:
gen("대출 문제로 여러 가지 힘드네.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'힘듦'